In [1]:
import numpy as np

def bankers_algorithm(processes, resources, instances, allocation, max_demand, requests):
    num_processes = len(processes)
    num_resources = len(resources)
    
    # Izračunavanje matrice potreba
    need = max_demand - allocation
    
    # Proveravanje da li je sistem u stabilnom stanju
    def is_safe(available, allocation, need):
        work = available.copy()
        finish = [False] * num_processes
        safe_sequence = []

        # Traženje sigurne sekvence
        while len(safe_sequence) < num_processes:
            allocated_this_round = False
            for p in range(num_processes):
                if not finish[p] and all(need[p] <= work):
                    work += allocation[p]
                    finish[p] = True
                    safe_sequence.append(processes[p])
                    allocated_this_round = True

            if not allocated_this_round:
                break
        
        # Provera da li je pronađena sigurna sekvenca
        return finish == [True] * num_processes, safe_sequence

    # Određivanje početnog stanja sistema
    available = instances - np.sum(allocation, axis=0)
    safe, sequence = is_safe(available, allocation, need)
    
    results = {
        "need_matrix": need,
        "is_safe": safe,
        "sequence": sequence,
        "requests": []
    }
    
    # Provera svakog zahteva
    for request in requests:
        p = processes.index(request[0])
        req = np.array(request[1:])
        
        if all(req <= need[p]) and all(req <= available):
            temp_available = available - req
            temp_allocation = allocation.copy()
            temp_allocation[p] += req
            temp_need = need.copy()
            temp_need[p] -= req
            
            temp_safe, _ = is_safe(temp_available, temp_allocation, temp_need)
            results["requests"].append((request, temp_safe))
        else:
            results["requests"].append((request, False))
    
    return results

def read_input_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Uklanjanje praznih linija i whitespace-ova
    lines = [line.strip() for line in lines if line.strip()]
    
    # Parsiranje podataka
    num_processes = int(lines[0].split(': ')[1])
    processes = lines[1].split(': ')[1].split()
    
    num_resources = int(lines[2].split(': ')[1])
    resources = lines[3].split(': ')[1].split()
    
    instances = np.array([int(x) for x in lines[4].split(': ')[1].split()])
    available = np.array([int(x) for x in lines[5].split(': ')[1].split()])

    allocation_start = 7
    allocation = np.array([[int(x) for x in lines[i].split()] for i in range(allocation_start, allocation_start + num_processes)])
    
    max_demand_start = allocation_start + num_processes + 1
    max_demand = np.array([[int(x) for x in lines[i].split()] for i in range(max_demand_start, max_demand_start + num_processes)])
    
    requests_start = max_demand_start + num_processes + 1
    requests = []
    for i in range(requests_start, len(lines)):
        req = lines[i].split()
        requests.append((req[0], int(req[1]), int(req[2]), int(req[3])))
    
    return processes, resources, instances, available, allocation, max_demand, requests

# Čitanje podataka iz datoteke i pokretanje algoritma
file_path = 'input.txt'  # putanja do vaše datoteke
processes, resources, instances, available, allocation, max_demand, requests = read_input_from_file(file_path)
results = bankers_algorithm(processes, resources, instances, allocation, max_demand, requests)

# Ispis rezultata
print("--- Rezultati simulacije ---")
print("Matrica potreba:")
print(results["need_matrix"])
print("Sistem je u stabilnom stanju" if results["is_safe"] else "Sistem nije u stabilnom stanju")
if results["is_safe"]:
    print("Sekvenca procesa:", "->".join(results["sequence"]))

for request, is_approved in results["requests"]:
    print(f"Zahtjev ({request[0]}, {request[1]}, {request[2]}, {request[3]}) {'će biti odobren' if is_approved else 'neće biti odobren'}.")

--- Rezultati simulacije ---
Matrica potreba:
[[7 4 3]
 [1 2 2]
 [6 0 0]
 [0 1 1]
 [4 3 1]]
Sistem je u stabilnom stanju
Sekvenca procesa: P2->P4->P5->P1->P3
Zahtjev (P1, 1, 1, 1) će biti odobren.
Zahtjev (P2, 3, 0, 0) neće biti odobren.
Zahtjev (P3, 3, 3, 1) neće biti odobren.
